In [ ]:
### IN THIS NOTEBOOK, I IMPORT THE GEOGRAPHICAL SURVEY DATA AND EXPORT AN HTML FILE CONTAINING AN INTERACTIVE MAP
### TO DISPLAY THIS DATA. 

In [2]:
### IMPORTING PACKAGES
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool, CategoricalColorMapper, LinearColorMapper, AdaptiveTicker, ColorBar
import geopandas as gpd
import pysal as ps
import pandas as pd
import numpy as np
import shapely
import matplotlib.pyplot as plt
import branca.colormap as cm

In [3]:
### LOADING FILES
ED_response_file_name='C:\\Users\\osman\\Downloads\\SD_18_response.shp'
#ED_totals_geojson_file_name = 'C:\\Users\\osman\\Downloads\\SD_18\\SD18_ED_totals.geojson'
#crime_gdf_file_name = 'C:\\Users\\osman\\Downloads\\crime_gdf.shp'

In [4]:
ED_response = gpd.read_file(ED_response_file_name)
ED_response.to_crs(epsg=4326);
#ED_totals['geoid'] = ED_totals.index.astype(str)

In [5]:
ED_response.head()

,Shape_Leng,Shape_Area,Nhood,HH,VOTERS,LATINOHH,JEWISHHH,LATINOPCT,JEWISHPCT,Yeses,Maybes,Nos,Complete,Targets,YesPct,MaybePct,NoPct,CompletePc,ADED,geometry
0,18577.466663,1.090507e+07,Greenpoint,59.0,62.0,7.0,14.0,0.11864,0.23728,5.0,6.0,0.0,11.0,62.0,0.45,0.54,0.00,0.17,50004,POLYGON ((-73.93729511262934 40.72930055678736...
1,4308.335941,8.388660e+05,Greenpoint,126.0,137.0,22.0,11.0,0.17460,0.08730,24.0,5.0,0.0,29.0,137.0,0.82,0.17,0.00,0.21,50005,POLYGON ((-73.95817589765223 40.73342568164895...
2,4827.746603,7.873451e+05,Greenpoint,85.0,94.0,12.0,14.0,0.14117,0.16470,7.0,8.0,0.0,15.0,94.0,0.46,0.53,0.00,0.15,50006,POLYGON ((-73.95484266604564 40.73374025901761...
3,2901.993411,4.840271e+05,Greenpoint,67.0,71.0,5.0,13.0,0.07462,0.19402,9.0,7.0,3.0,19.0,71.0,0.47,0.36,0.15,0.26,50007,POLYGON ((-73.95472720538594 40.73303092801621...
4,4354.875174,6.205534e+05,Greenpoint,85.0,90.0,14.0,10.0,0.16470,0.11764,16.0,8.0,0.0,24.0,90.0,0.66,0.33,0.00,0.26,50008,POLYGON ((-73.95189485181066 40.73186185613005...


In [ ]:
### CALCULATING THE NUMBER OF NON-NEGATIVE CONTACTS (A MAYBE OR A YES) BY DISTRICT

In [16]:
ED_response['Remaining'] = ED_response['Targets'] - ED_response['Complete']

In [19]:
ED_response['NonNegPct'] = 1 - ED_response['NoPct'] 

In [21]:
ED_response['NonNegContacts'] =  ED_response['NonNegPct']*ED_response['Remaining']

In [24]:
### SOME DATA VALIDATION
((ED_response['Remaining'] - ED_response['NonNegContacts']) < 0).value_counts()

False    244
dtype: int64

In [25]:
###NUMBER OF NON-NEGATIVE CONTACTS - SORTING BY THIS COLUMN CAN TELL US WHERE WE HAVE THE MOST POTENTIAL IN A SENSE
ED_response['NonNegContacts']

0       51.00
1      108.00
2       79.00
3       44.20
4       66.00
5       51.52
6       70.00
7       68.00
8       87.00
9       77.00
10      74.00
11      67.76
12      87.42
13      49.00
14       0.00
15      15.12
16      47.00
17      71.00
18      14.00
19      68.04
20      45.00
21      90.00
22      98.00
23     127.00
24      63.00
25      39.00
26      24.00
27      83.00
28     111.00
29      52.00
        ...  
214      9.00
215      2.00
216      0.00
217     29.23
218     16.00
219     41.00
220     69.00
221     33.00
222     32.01
223     60.00
224     38.00
225     84.60
226     67.00
227     33.00
228      0.00
229     13.00
230     27.00
231      6.00
232     22.00
233     11.00
234     51.00
235     30.00
236    101.00
237     68.00
238     40.00
239     91.00
240     89.00
241     23.00
242     18.00
243     80.00
Name: NonNegContacts, Length: 244, dtype: float64

In [6]:
### IMPORTING PACKAGES FOR FOLIUM EXPORT
from geopandas.tools import sjoin
import folium
from folium.plugins import MarkerCluster
from folium import IFrame
import shapely
from shapely.geometry import Point
import unicodedata
import pysal as ps

In [28]:
### DEFINING FOLIUM MAP AND LOOPING OVER ROWS IN DATASET TO BUILD MAP 
m = folium.Map(location=[40.701782, -73.930189], zoom_start=13,
               tiles='Stamen Toner'
              )

fg2 = folium.map.FeatureGroup(name='Expected # of non-negative contacts:')
#cmap1 = folium.LinearColormap(['white','blue'], vmin=0, vmax=ED_totals['bin_geocli'].max(), caption='number')
colormap = cm.linear.BuPu_09
cmap1 = colormap.scale(0, ED_response['NonNegContacts'].max())
for i,r in ED_response.iterrows():
    gjson = ED_response[ED_response['ADED']==r.ADED].to_json()
    rgbv = cmap1(r['NonNegContacts'])
    gj = folium.GeoJson(gjson, style_function = lambda feature, rgbv=rgbv: {
            'fillColor': rgbv,
            'color' : 'black',
            'weight' : 0.5,
            'fillOpacity' : 0.7,
    })
    folium.Popup('<b>ED {} </b><br /><b>Number of Maybes:</b> {}<br /><b>Complete %:</b> {}<br /><b>Number of Targets:</b> {}<br /><b>Expected Number of Non-Negative Contacts:</b> {}<br />'.format(float(r['ADED']), int(r['Maybes']), 100*float(r['CompletePc']), int(r['Targets']), int(r['NonNegContacts']) )).add_to(gj)
    gj.add_to(fg2)
                     
fg2.add_to(m)


folium.LayerControl().add_to(m)
m.save('ED_response_web.html')